# Transformer
- 구글이 발표한 [Attention is all you need](https://arxiv.org/pdf/1706.03762) 논문에서 제안된 모델입니다.
  - Attention은 17에서 다루었듯, Seq2Seq의 단점을 보완하기 위한 구조였는데, 해당 논문에서는 Attention만을 이용해서 제작한 모델을 소개합니다.

## Seq2Seq의 한계
- Seq2Seq은 Encoder를 이용하여 Sequence를 하나의 Context Vector로 압축하여 표현하고, Context Vector를 이용하여 Output Sequence를 제작합니다.
  - 이때, Encoder에서 하나의 벡터로 압축하며, 정보의 손실이 발생하기에 long sequence에서는 문제가 존재했습니다.
  - 이것을 보정하기 위해 Attention Mechanism이 도입되었습니다.
- 그렇다면, **Attention만으로 Encoder, Decoder를 구성하면 어떨까?** 이것이 Transformer의 시작입니다.


## Hyperparameter of Transformer
- $d_{model}$
  - Encoder, Decoder에서 정해진 차원이며, 임베딩 벡터의 차원입니다.
  - 각 Encoder, Decoder가 다음 Layer의 Encoder Decoder로 값을 보낼 때도 이 Dimension을 유지합니다.
  - 논문에서는 512를 사용했습니다.
- *num_layers*
  - 각 층은 Encoder와 Decoder로 이루어지는데, 이 층을 몇 개로 설정할지에 관한 파라미터입니다.
  - 논문에서는 6을 사용했습니다.
- *num_heads*
  - Transformer 논문에서는 Attetion을 한 번 하는 것이 아닌 병렬로 연결해서 결과를 합치는 방식으로 이용합니다.
  - num_heads는 이때, 병렬로 이용할 Attention의 개수입니다.
- $d_{ff}$
  - Transformer 내부의 FeedForwardNeuralNetwork의 크기를 나타냅니다.
  - 신경망의 입력층과 출력층의 크기는 위에서 언급하였던 $d_{model}$입니다.

## Structure of Transformer
- Seq2Seq
  - Encoder와 Decoder에서 각각 하나의 RNN이 t개의 시점을 가지고 동작합니다.
- Transformer
  - Encoder와 Decoder가 N개로 구성되어 동작합니다.

## Input of Transformer
- Transformer의 Input은 Seq2Seq과 다르게 각 단어의 임베딩을 직접 받는 것이 아닌, 조정된 임베딩 벡터를 Input으로 받습니다.
  - Transformer는 RNN과 다르게 순차적으로 입력을 받는 것이 아닌 병렬적으로 동작합니다.
  - 따라서, Transformer에서는 각 단어의 위치 관계를 다른 방식으로 전달해줄 필요가 있습니다.
  - 이때 사용하는 방법이 임베딩 벡터에 위치 정보를 더하여 전달해주는 것이고
    - 이것을 **Positional Encodeing**이라고 합니다.
### Positional Encoding
- Positional Encoding이 위치정보를 반영하게 해야하므로 논문에서는 아래와 같은 식을 사용하였습니다.
  - $PE_{pos,2i} = sin(\frac{pos}{{10000}^{ 2i /d_{model}}})$
  - $PE_{pos,2i+1} = cos(\frac{pos}{{10000}^{ 2i /d_{model}}})$
  - $pos$
    - 입력 문장 안에서 임베딩 벡터의 위치
  - $i$
    - 임베딩 벡터 내의 차원의 인덱스
  - $d_{model}$
    - 모든 층에서 입,출력의 차원
    - 임베딩 벡터도 $d_{model}$의 차원을 가집니다.

## Transformer's three attentions

### Encoder Self-Attention
- Encoder에서 이루어지는 Self-Attention

### Masked Decoder Self Attention
- Decoder에서 이루어지는 Self-Attention

### Encoder Decoder Attention
- Decoder에서 이루어지는 Attention
  - Self-Attention이 아닌 이유
    - Self-Attention은 Q,K,V가 모두 출처가 동일한 상황에서의 Attention을 말합니다.
    - 하지만, Encoder Decoder Attention에서는 Query가 Decoder의 벡터, Key,Value가 Encoder의 벡터이므로 Self-Attention이라고 하지 않습니다.


## Encoder
- Encoder의 구조는 아래와 같습니다.
  - Input - Embedding - Positional Encodeing - Multi-head self attention - Add&Norm - FFNN - Add&Norm
  - 이러한 Encoder가 N개의 층으로 이루어집니다.

### Self-Attention
-  Attention
  - Query에 대해 모든 Key와의 유사도를 구합니다.
  - 해당 유사도를 가중치로 하여 Key와 매핑 되어있는 각 Value에 반영해준 후, 가중합하여 최종 output을 Return합니다.
- Self-Attention
  -

In [ ]:
ㅇ